# AI4Code Baseline Model

This notebook implements a baseline solution for the AI4Code competition.

## Approach
1. Extract basic features from cells (length, type, etc.)
2. Train a model to predict cell ordering
3. Use Kendall tau correlation as evaluation metric

In [1]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import re
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Set paths
DATA_PATH = Path('/home/data')
TRAIN_PATH = DATA_PATH / 'train'
TEST_PATH = DATA_PATH / 'test'
TRAIN_ORDERS_PATH = DATA_PATH / 'train_orders.csv'

print("Loading data paths...")
print(f"Train path: {TRAIN_PATH}")
print(f"Test path: {TEST_PATH}")
print(f"Train orders: {TRAIN_ORDERS_PATH}")

Loading data paths...
Train path: /home/data/train
Test path: /home/data/test
Train orders: /home/data/train_orders.csv


In [2]:
# Load train orders
train_orders = pd.read_csv(TRAIN_ORDERS_PATH)
print(f"Train orders shape: {train_orders.shape}")
print(f"First few rows:")
print(train_orders.head())

Train orders shape: (119256, 2)
First few rows:
               id                                         cell_order
0  00001756c60be8  1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 7...
1  0001daf4c2c76d  97266564 a898e555 86605076 76cc2642 ef279279 d...
2  0002115f48f982  9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a...
3  00035108e64677  3496fbfe 2fa1f27b 719854c4 f3c2de19 d75feb42 5...
4  00038c2941faa0  3e551fb7 45049ad8 8bb41691 123b4f4c 0b92cb59 5...


In [3]:
# Load a sample notebook to understand structure
def load_notebook(notebook_id, path):
    """Load a notebook from JSON file"""
    with open(path / f"{notebook_id}.json", 'r') as f:
        notebook = json.load(f)
    return notebook

# Load sample notebook
sample_nb = load_notebook('0002115f48f982', TRAIN_PATH)
print("Notebook keys:", list(sample_nb.keys()))
print("\nCell types:", list(sample_nb['cell_type'].items())[:5])
print("\nSample code cell:", list(sample_nb['source'].items())[0])
print("\nSample markdown cell:", list(sample_nb['source'].items())[-1])

Notebook keys: ['cell_type', 'source']

Cell types: [('18281c6c', 'code'), ('e3b6b115', 'code'), ('4a044c54', 'code'), ('365fe576', 'code'), ('a3188e54', 'code')]

Sample code cell: ('18281c6c', 'import numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n%matplotlib inline\nimport os\nprint(os.listdir("../input"))\n')

Sample markdown cell: ('9ec225f0', 'Hi there,\n\nIs it ok that the same measurement have different target labels between signals?\nAccording to data description it should be the same (or not really?) . There are 38 cases of measurements with not consistent labels between signals.\n\nQuick and dirty code to show the problem below:')


In [4]:
# Create features for a notebook
def extract_features(notebook_id, path):
    """Extract features from a notebook"""
    notebook = load_notebook(notebook_id, path)
    
    features = []
    cell_ids = []
    
    for cell_id, cell_type in notebook['cell_type'].items():
        source = notebook['source'][cell_id]
        
        # Basic features
        feature_dict = {
            'notebook_id': notebook_id,
            'cell_id': cell_id,
            'cell_type': cell_type,
            'source_length': len(source),
            'line_count': source.count('\n') + 1,
            'word_count': len(source.split()),
            'char_count': len(source.replace('\n', '').replace(' ', '')),
            'has_import': int('import ' in source or 'from ' in source) if cell_type == 'code' else 0,
            'has_comment': int('#' in source) if cell_type == 'code' else 0,
            'has_heading': int(any(heading in source for heading in ['# ', '## ', '### '])) if cell_type == 'markdown' else 0,
            'has_code_block': int('```' in source) if cell_type == 'markdown' else 0,
            'has_link': int('http' in source or 'www.' in source) if cell_type == 'markdown' else 0,
        }
        
        features.append(feature_dict)
        cell_ids.append(cell_id)
    
    return pd.DataFrame(features), cell_ids

# Test feature extraction
sample_features, sample_cell_ids = extract_features('0002115f48f982', TRAIN_PATH)
print("Sample features shape:", sample_features.shape)
print(sample_features.head())

Sample features shape: (9, 12)
      notebook_id   cell_id cell_type  source_length  line_count  word_count  \
0  0002115f48f982  18281c6c      code            220           8          32   
1  0002115f48f982  e3b6b115      code             57           2           4   
2  0002115f48f982  4a044c54      code              9           1           1   
3  0002115f48f982  365fe576      code            164           3          15   
4  0002115f48f982  a3188e54      code            119           2          18   

   char_count  has_import  has_comment  has_heading  has_code_block  has_link  
0         188           1            1            0               0         0  
1          54           0            0            0               0         0  
2           9           0            0            0               0         0  
3         150           0            1            0               0         0  
4         101           0            1            0               0         0  


In [5]:
# Create training data
def create_training_data(notebook_ids, path, orders_df):
    """Create training dataset with features and target positions"""
    all_features = []
    all_targets = []
    
    for notebook_id in tqdm(notebook_ids, desc="Processing notebooks"):
        # Extract features
        features, cell_ids = extract_features(notebook_id, path)
        
        # Get correct order
        correct_order = orders_df[orders_df['id'] == notebook_id]['cell_order'].iloc[0].split()
        
        # Create position mapping
        position_map = {cell_id: pos for pos, cell_id in enumerate(correct_order)}
        
        # Add target position
        features['position'] = features['cell_id'].map(position_map)
        
        all_features.append(features)
    
    return pd.concat(all_features, ignore_index=True)

# Load a subset for initial training
sample_notebooks = train_orders['id'].head(1000).tolist()
train_df = create_training_data(sample_notebooks, TRAIN_PATH, train_orders)
print(f"Training data shape: {train_df.shape}")
print(train_df.head())

Processing notebooks:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing notebooks:   1%|          | 8/1000 [00:00<00:13, 74.18it/s]

Processing notebooks:   2%|▏         | 16/1000 [00:00<00:13, 73.08it/s]

Processing notebooks:   2%|▏         | 24/1000 [00:00<00:13, 73.15it/s]

Processing notebooks:   3%|▎         | 32/1000 [00:00<00:12, 75.14it/s]

...

Processing notebooks:  98%|█████████▊| 978/1000 [00:11<00:00, 90.16it/s]

Processing notebooks:  99%|█████████▉| 988/1000 [00:11<00:00, 90.77it/s]

Processing notebooks: 100%|█████████▉| 998/1000 [00:11<00:00, 89.84it/s]

Processing notebooks: 100%|██████████| 1000/1000 [00:11<00:00, 84.16it/s]

Training data shape: (46232, 13)
      notebook_id   cell_id cell_type  source_length  line_count  word_count  \
0  00001756c60be8  1862f0a6      code            930          17         140   
1  00001756c60be8  2a9e43d6      code            498          17          55   
2  00001756c60be8  038b763d      code             49           2           3   
3  00001756c60be8  2eefe0ef      code             45           1           2   
4  00001756c60be8  0beab1cd      code            694          19          39   

   char_count  has_import  has_comment  has_heading  has_code_block  has_link  \
0         774           1            1            0               0         0   
1         441           1            0            0               0         0   
2          47           1            0            0               0         0   
3          44           0            0            0               0         0   
4         584           0            0            0               0         0   


In [6]:
# Prepare features for modeling
feature_cols = [col for col in train_df.columns if col not in ['notebook_id', 'cell_id', 'position', 'cell_type']]
print(f"Feature columns: {feature_cols}")

# Add cell type as categorical feature
train_df['cell_type_code'] = (train_df['cell_type'] == 'code').astype(int)
feature_cols.append('cell_type_code')

print(f"Final feature columns: {feature_cols}")
print(f"Training data shape: {train_df.shape}")

Feature columns: ['source_length', 'line_count', 'word_count', 'char_count', 'has_import', 'has_comment', 'has_heading', 'has_code_block', 'has_link']
Final feature columns: ['source_length', 'line_count', 'word_count', 'char_count', 'has_import', 'has_comment', 'has_heading', 'has_code_block', 'has_link', 'cell_type_code']
Training data shape: (46232, 14)


In [7]:
# Define Kendall tau metric for evaluation
def kendall_tau_score(y_true, y_pred):
    """Calculate Kendall tau correlation"""
    from scipy.stats import kendalltau
    return kendalltau(y_true, y_pred).correlation

kendall_scorer = make_scorer(kendall_tau_score, greater_is_better=True)

# Train model
print("Training LightGBM model...")

# Use a subset for faster training
sample_df = train_df.sample(frac=0.3, random_state=42)

X = sample_df[feature_cols]
y = sample_df['position']

# Train LightGBM model
model = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    n_jobs=-1
)

model.fit(X, y)

print("Model training completed!")
print(f"Feature importances: {dict(zip(feature_cols, model.feature_importances_))}")

Training LightGBM model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001205 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 852
[LightGBM] [Info] Number of data points in the train set: 13870, number of used features: 10
[LightGBM] [Info] Start training from score 39.902451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

In [8]:
# Predict on test data
def predict_notebook_order(notebook_id, path, model, feature_cols):
    """Predict cell order for a notebook"""
    features, cell_ids = extract_features(notebook_id, path)
    
    # Add cell type code
    features['cell_type_code'] = (features['cell_type'] == 'code').astype(int)
    
    # Predict positions
    X = features[feature_cols]
    predicted_positions = model.predict(X)
    
    # Sort by predicted position
    order_df = pd.DataFrame({
        'cell_id': cell_ids,
        'predicted_position': predicted_positions
    })
    
    # Sort and get ordered cell IDs
    ordered_cells = order_df.sort_values('predicted_position')['cell_id'].tolist()
    
    return ' '.join(ordered_cells)

# Test on a few notebooks
test_notebooks = [f.stem for f in TEST_PATH.glob('*.json')][:5]
print("Testing prediction on sample notebooks:")
for nb_id in test_notebooks:
    try:
        predicted_order = predict_notebook_order(nb_id, TEST_PATH, model, feature_cols)
        print(f"{nb_id}: {predicted_order[:100]}...")
    except Exception as e:
        print(f"Error with {nb_id}: {e}")

Testing prediction on sample notebooks:
a283f88578d447: 28dd0afc 2f5e2550 971426cb 29afef3c ff11e8eb 1e9b9872 8e1b86b4 65d5a63b c4ce3543 ee231d0b 695f2f51 4...


4b5781d84d4f04: 1898e18c def9cf6d c45bd162 2ebababb e526a036 f58bfff9 0a3111b8 6b9348fb 2deaa157 1b5b58e3 cfce3ff7 a...
032e1e8598af69: 5f0c5c07 b900c928 2f301a68 a2ea3320 c906e98e dc8bb19d 97fe5057 65a5abac 2b10ddf8 4c841a98 5aaa2adb f...
598969c8543546: 8115e5d5 ff8abe04 71472504 a3c36e84 8b27b109 31e67a94 de42315e 4d9eb115 15bcdeea ca7d7654 2c952d06 b...


6986f993569b86: 9ce5d5e0 04e4e828 b4b4c96e 8cafb9e2 b76ce247 e3bf698e b1ddce4d 0eed5fa5 222473db 7260068e 86c5ffb8 7...


In [9]:
# Generate submission for all test notebooks
print("Generating submission for all test notebooks...")

test_notebooks = [f.stem for f in TEST_PATH.glob('*.json')]
submission_data = []

for notebook_id in tqdm(test_notebooks, desc="Predicting notebooks"):
    try:
        predicted_order = predict_notebook_order(notebook_id, TEST_PATH, model, feature_cols)
        submission_data.append({
            'id': notebook_id,
            'cell_order': predicted_order
        })
    except Exception as e:
        print(f"Error with {notebook_id}: {e}")
        # Use default order (code cells first, then markdown cells in random order)
        submission_data.append({
            'id': notebook_id,
            'cell_order': ''
        })

submission_df = pd.DataFrame(submission_data)
print(f"Submission shape: {submission_df.shape}")
print(submission_df.head())

Generating submission for all test notebooks...


Predicting notebooks:   0%|          | 0/20000 [00:00<?, ?it/s]

Predicting notebooks:   0%|          | 23/20000 [00:00<01:29, 223.12it/s]

Predicting notebooks:   0%|          | 46/20000 [00:00<01:29, 224.07it/s]

Predicting notebooks:   0%|          | 72/20000 [00:00<01:23, 238.65it/s]

...

Predicting notebooks: 100%|█████████▉| 19936/20000 [01:35<00:00, 233.92it/s]

Predicting notebooks: 100%|█████████▉| 19960/20000 [01:35<00:00, 225.92it/s]

Predicting notebooks: 100%|█████████▉| 19983/20000 [01:35<00:00, 215.23it/s]

Predicting notebooks: 100%|██████████| 20000/20000 [01:35<00:00, 209.31it/s]

Submission shape: (20000, 2)
               id                                         cell_order
0  a283f88578d447  28dd0afc 2f5e2550 971426cb 29afef3c ff11e8eb 1...
1  4b5781d84d4f04  1898e18c def9cf6d c45bd162 2ebababb e526a036 f...
2  032e1e8598af69  5f0c5c07 b900c928 2f301a68 a2ea3320 c906e98e d...
3  598969c8543546  8115e5d5 ff8abe04 71472504 a3c36e84 8b27b109 3...
4  6986f993569b86  9ce5d5e0 04e4e828 b4b4c96e 8cafb9e2 b76ce247 e...


In [10]:
# Save submission
submission_path = '/home/submission/submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"Submission saved to {submission_path}")

# Verify format
sample_sub = pd.read_csv('/home/data/sample_submission.csv')
print(f"\nSample submission format:")
print(sample_sub.head())
print(f"\nOur submission format:")
print(submission_df.head())

print(f"\nColumns match: {list(submission_df.columns) == list(sample_sub.columns)}")
print(f"Number of rows match: {len(submission_df) == len(sample_sub)}")

Submission saved to /home/submission/submission.csv

Sample submission format:
               id                                         cell_order
0  00015c83e2717b  c417225b 51e3cd89 2600b4eb 75b65993 cf195f8b 2...
1  0001bdd4021779  3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 0...
2  000757b90aaca0  8f84d7a9 eb6ca769 bc595bc2 93cceeef 3cb3d383 6...
3  000a2f5243e1ca  1d968d84 5774aca9 2ddb979d dc18005d 3c7d2db1 2...
4  000c1e0e45bb25  3255fff2 d6c42868 dd65465b 07c5250a 31aafb08 8...

Our submission format:
               id                                         cell_order
0  a283f88578d447  28dd0afc 2f5e2550 971426cb 29afef3c ff11e8eb 1...
1  4b5781d84d4f04  1898e18c def9cf6d c45bd162 2ebababb e526a036 f...
2  032e1e8598af69  5f0c5c07 b900c928 2f301a68 a2ea3320 c906e98e d...
3  598969c8543546  8115e5d5 ff8abe04 71472504 a3c36e84 8b27b109 3...
4  6986f993569b86  9ce5d5e0 04e4e828 b4b4c96e 8cafb9e2 b76ce247 e...

Columns match: True
Number of rows match: True
